In [23]:
import pandas as pd
import numpy as np
import json
import os
import six
from collections import Counter
from docx import Document
import os
import re
import pdb
import re
from random import choice
import random
import torch
from transformers import AutoTokenizer, BertModel
from datasets import Dataset
import torch
torch.cuda.is_available()


True

In [2]:
path1 = '' #待读取文件的文件夹绝对地址
# path1 = './demo_地级市工作报告' #待读取文件的文件夹绝对地址
files = os.listdir(path1) #获得文件夹中所有文件的名称列表
test_dataset = pd.DataFrame(
  columns=['text','city','year']
) 
for file in files:
    try:
        city_name = file[:-9]
        year_name = int(file[-9:-5])
        path = path1 +  '/' + file
        document = Document(path)
        sentences = document.paragraphs[0].text.strip().replace('\n','').split('。')
        for sentence in sentences:
            if any(keyword in sentence for keyword in keyword_list):
                test_dataset.loc[len(test_dataset.index)] = [sentence, city_name, year_name]
            # break
    except:
        pass 
test_dataset.to_csv("0414res/test_dataset/test_dataset.csv", encoding="utf-8")

In [2]:
test_dataset = pd.read_csv("0414res/test_dataset/test_dataset.csv", encoding="utf-8", index_col=0)
table_1 = pd.read_csv("")
pos_table = table_1[table_1[""] == 1]
neg_table = table_1[table_1[""] == 0]
pos_list = pos_table["text"].tolist()
neg_list = neg_table["text"].tolist()
def set_lable(text):
    if any(neg_word in text for neg_word in neg_list):
        return 0
    if any(pos_word in text for pos_word in pos_list):
        return 1
    return -1
test_dataset["lable"] = test_dataset.apply(lambda x: set_lable(x.text), axis=1)
train_dataset = test_dataset[test_dataset["lable"] != -1]
train_dataset.reset_index(drop=True)
train_dataset.to_csv("0414res/train_dataset/train_dataset.csv", encoding="utf-8")
print(test_dataset["lable"].value_counts())
train_dataset


 1    231653
 0     56254
-1     34404
Name: lable, dtype: int64


,text,city,year,lable
0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0
1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,1
2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,1
3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,1
4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,1
...,...,...,...,...
322305,此外，加快推进科技创新、社会保障、医疗卫生、文化体制等领域改革，增强发展的内生动力,吉林省白城市,2018,1
322306,抓住国家实施区域协调发展战略机遇，争取把白齐兴跨省生态经济合作区上升为国家战略，加强省际协商...,吉林省白城市,2018,1
322307,（4）深入推进文化惠民工程，滑冰馆、体育场、游泳馆投入使用，贫困村综合性文化服务中心实现全覆盖,吉林省白城市,2018,1
322308,（5）加大义务教育薄弱学校改造，建设标准化学校5所，实施校舍取暖“煤改电”,吉林省白城市,2018,0


In [3]:
import torch
from datasets import load_dataset
from transformers import BertTokenizer, BertModel, AutoTokenizer
#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')
pretrained = BertModel.from_pretrained('bert-base-chinese')

#定义数据集

# dataset = load_dataset(path='csv',
#                            data_files=r'0414res/train_dataset/train_dataset.csv',
#                            split='train')

dataset = load_dataset(path='csv',
                           data_files='./0414res/train_dataset/train_dataset.csv',
                           split='train')


def collate_fn(data):
    sents = [i["text"] for i in data]
    labels = [i["lable"] for i in data]
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']
    labels = torch.LongTensor(labels)
    return input_ids, attention_mask, token_type_ids, labels
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=64,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for param in pretrained.parameters():
    param.requires_grad_(False)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Using custom data configuration default-f12c7aae6252ad12
Reusing dataset csv (/home/leo/.cache/huggingface/datasets/csv/default-f12c7aae6252ad12/0.0.0/652c

In [4]:
print(torch.cuda.device_count())
#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out

model = Model()


1


In [16]:
from transformers import AdamW
device = torch.device("cuda")	
pretrained = pretrained.to(device)
model = model.to(device)
#训练
optimizer = AdamW(model.parameters(), lr=5e-4)
criterion = torch.nn.CrossEntropyLoss()
criterion  = criterion.cuda()

for j in range(5):
    model.train()
    for i, (input_ids, attention_mask, token_type_ids,
            labels) in enumerate(loader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        labels = labels.to(device)
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids)
        out = out.to(device)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 500 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print(i, loss.item(), accuracy)
        if i % 1000 == 0:
            torch.save(model.state_dict(), 'models/model_state_dict_{}epoch_{}iter.pkl'.format(j, i))
        if i == 4400:
            torch.save(model.state_dict(), 'models/model_state_dict_{}epoch_{}iter.pkl'.format(j, i))
            break

/home/leo/anaconda3/envs/ernie/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


0 0.7536423802375793 0.328125
500 0.45232871174812317 0.859375
1000 0.4395332336425781 0.875
1500 0.47786781191825867 0.796875
2000 0.49358224868774414 0.796875
2500 0.4533219635486603 0.859375
3000 0.447574645280838 0.859375
3500 0.4980705678462982 0.78125
4000 0.5513716340065002 0.75
0 0.4366416931152344 0.859375
500 0.46221262216567993 0.84375
1000 0.4701763391494751 0.828125
1500 0.5007040500640869 0.8125
2000 0.48027586936950684 0.828125
2500 0.5182220339775085 0.796875
3000 0.4265836775302887 0.875
3500 0.466220498085022 0.828125
4000 0.43990710377693176 0.875
0 0.46093252301216125 0.859375
500 0.4843500256538391 0.828125
1000 0.4970298707485199 0.8125
1500 0.4512084424495697 0.859375
2000 0.41311970353126526 0.90625
2500 0.5012664198875427 0.796875
3000 0.4965287446975708 0.796875
3500 0.4200526475906372 0.890625
4000 0.48926854133605957 0.796875
0 0.490387499332428 0.828125
500 0.45792102813720703 0.84375
1000 0.4773271977901459 0.828125
1500 0.47736719250679016 0.8125
2000 0.4

In [55]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
# model = BertModel.from_pretrained("bert-base-chinese")
model_for_test = Model()
model_for_test.load_state_dict(torch.load('models/model_state_dict_4epoch_4400iter.pkl'))
dataset_val = pd.read_csv("./0414res/test_dataset/test_dataset.csv", index_col=0)

In [58]:
test_text = dataset_val.loc[ 1, "text"]  

device = torch.device("cuda")	
# device = torch.device("cpu")	
model_for_test = model_for_test.to(device)
pretrained = pretrained.to(device)
def get_lable_res(test_text):
    test_text = test_text[:510] # tokenizer 不提供截断padding接口，人为截断
    inputs = tokenizer(test_text, return_tensors="pt")
    res = model_for_test(input_ids=inputs["input_ids"].to(device), 
                        attention_mask=inputs["attention_mask"].to(device), 
                        token_type_ids=inputs["token_type_ids"].to(device))
    return res.cpu().detach().numpy()[0][0], res.cpu().detach().numpy()[0][1], res.argmax(dim=1).cpu().detach().numpy()[0]
get_lable_res(test_text)

(0.015324044, 0.98467594, 1)

In [59]:
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas(desc='set new lable')

dataset_val_1 = dataset_val.head(1000)
dataset_val_1["neg_prob"], dataset_val_1["pos_prob"], dataset_val_1["lable"] = zip(*dataset_val_1.parallel_apply(lambda x: get_lable_res(x.text), axis=1))
# dataset_val["neg_prob"], dataset_val["pos_prob"], dataset_val["lable"] = zip(*dataset_val.parallel_apply(lambda x: get_lable_res(x.text), axis=1))
# dataset_val.to_csv("0414res/trained_res.csv", encoding="utf-8")
dataset_val_1.to_csv("0414res/trained_res.csv", encoding="utf-8")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

KeyboardInterrupt: 

In [52]:
! pip install pandarallel

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: /home/leo/anaconda3/envs/ernie/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py) ... done
  Created wheel for pandarallel: filename=pandarallel-1.6.4-py3-none-any.whl size=16662 sha256=e14e58e2d389d9daa39ece27844f9b361ae4ae5842cf492137aeee9d489031e7
  Stored in directory: /home/leo/.cache/pip/wheels/51/00/2a/5518392a70f9d8fd2a48794080649c0ea9d03d39974d2be0e3
Successfully built pandarallel


In [53]:
from pandarallel import pandarallel
 
# Initialization
pandarallel.initialize()
 
# # Standard pandas apply
# df.apply(func)
 
# # Parallel apply
# df.parallel_apply(func)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
import pandas as pd
import numpy as np
import json
import os
import six
from collections import Counter
from docx import Document
import os
import re
import pdb
import re
from random import choice
import random
import torch
from transformers import AutoTokenizer, BertModel
from datasets import Dataset
import torch
import time
torch.cuda.is_available()
from tqdm import tqdm

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
pretrained = BertModel.from_pretrained("bert-base-chinese")
model_for_test = Model()
model_for_test.load_state_dict(torch.load('models/model_state_dict_4epoch_4400iter.pkl'))
dataset_val = pd.read_csv("./0414res/test_dataset/test_dataset.csv", index_col=0)


device = torch.device("cuda")	
model_for_test = model_for_test.to(device)
pretrained = pretrained.to(device)

def get_lable_res(test_text):
    test_text = test_text[:510] # tokenizer 不提供截断padding接口，人为截断
    inputs = tokenizer(test_text, return_tensors="pt")
    res = model_for_test(input_ids=inputs["input_ids"].to(device), 
                        attention_mask=inputs["attention_mask"].to(device), 
                        token_type_ids=inputs["token_type_ids"].to(device))
    return res.cpu().detach().numpy()[0][0], res.cpu().detach().numpy()[0][1], res.argmax(dim=1).cpu().detach().numpy()[0]

tqdm.pandas(desc='set new lable')
a = time.time()
# dataset_val_1["neg_prob"], dataset_val_1["pos_prob"], dataset_val_1["lable"] = zip(*dataset_val_1.parallel_apply(lambda x: get_lable_res(x.text), axis=1))
dataset_val["neg_prob"], dataset_val["pos_prob"], dataset_val["lable"] = zip(*dataset_val.progress_apply(lambda x: get_lable_res(x.text), axis=1))
print(time.time() - a, ": gpu 用时")


# dataset_val["neg_prob"], dataset_val["pos_prob"], dataset_val["lable"] = zip(*dataset_val.parallel_apply(lambda x: get_lable_res(x.text), axis=1))
dataset_val.to_csv("0414res/trained_res.csv", encoding="utf-8")
# dataset_val_1.to_csv("0414res/trained_res.csv", encoding="utf-8")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
set new lable: 100%|██████████| 322311/322311 [36:54<00:00, 145.56it/s]


2215.2852737903595 : gpu 用时


In [4]:
from tqdm import tqdm,trange,tnrange,tqdm_notebook
from time import sleep

# 普通进度条
for i in trange(60):
	#TODO:
	sleep(.1)

# 专为notebook设计的进度条
for i in tqdm_notebook(range(600)):
	#TODO:
	sleep(.1)

tqdm.pandas(desc='pandas bar')
df_result = df['score'].progress_apply(lambda x: str(x)[0:7])

100%|██████████| 60/60 [00:06<00:00,  9.89it/s]
/tmp/ipykernel_5510/610718300.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(600)):


  0%|          | 0/600 [00:00<?, ?it/s]

In [12]:
dataset_val.head(10)

,text,city,year,neg_prob,pos_prob,lable
0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0.007142,0.992858,1
1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,0.015324,0.984676,1
2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,0.005322,0.994678,1
3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,0.000033,0.999967,1
4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,0.000090,0.999910,1
5,我市被评为“中国特色养生休闲旅游最佳目的地”,广东省云浮市,2016,0.000021,0.999979,1
6,全年接待游客2062.59万人次，增长9.9% ，实现社会旅游总收入214.59亿元，增长9...,广东省云浮市,2016,0.000173,0.999827,1
7,南广铁路交通枢纽、中央商务区、北湖启动区等项目有序推进,广东省云浮市,2016,0.000036,0.999964,1
8,云安区的商贸金融、文化教育、医疗卫生、居住、行政服务等功能加快向西江新城集中,广东省云浮市,2016,0.040693,0.959307,1
9,市中心城区公共服务设施进一步完善，河滨西路、金丰路、金山路（北段）等3条市政道路全线贯通,广东省云浮市,2016,0.501274,0.498726,0


In [16]:
dataset_val_mean = dataset_val[["city", "year", "lable"]]
res = dataset_val_mean.groupby(["city", "year"]).mean()['lable']

In [21]:
# res.to_csv("demo.csv", encoding="utf-8")
# res = pd.read_csv("demo.csv")
new = res.sort_values(by=['lable'],ascending=[False])

In [25]:
new.head(50)

,city,year,lable
406,四川省乐山市,2010,1.000000
3958,陕西省宝鸡市,2020,1.000000
1430,广东省中山市,2017,1.000000
1981,江苏省南通市,2020,1.000000
3003,湖北省荆州市,2017,1.000000
1427,广东省中山市,2014,1.000000
1868,广西壮族自治区百色市,2020,1.000000
950,安徽省铜陵市,2014,1.000000
1422,广东省中山市,2009,1.000000
3922,陕西省商洛市,2013,1.000000


In [28]:
dataset_val_mean = dataset_val[["city", "year", "pos_prob"]]
res1 = dataset_val_mean.groupby(["city", "year"]).mean()['pos_prob']
res1.to_csv("demo1.csv", encoding="utf-8")
res1 = pd.read_csv("demo1.csv")
new1 = res1.sort_values(by=['pos_prob'],ascending=[True])
new1.head(50)

,city,year,pos_prob
4110,黑龙江省双鸭山市,2007,0.576785
4146,黑龙江省大庆市,2013,0.633674
1953,江苏省南京市,2007,0.653025
4148,黑龙江省大庆市,2015,0.653665
4168,黑龙江省牡丹江市,2020,0.660477
2221,江西省抚州市,2008,0.662660
1840,广西壮族自治区玉林市,2007,0.669512
3148,湖南省张家界市,2013,0.671580
195,内蒙古自治区呼和浩特市,2008,0.679614
2888,海南省三亚市,2007,0.683542


In [2]:
import pandas as pd
import numpy as np

file_demo = pd.read_csv("0414res/trained_res.csv")
file_demo

,Unnamed: 0,text,city,year,neg_prob,pos_prob,lable
0,0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0.007142,0.992858,1
1,1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,0.015324,0.984676,1
2,2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,0.005322,0.994678,1
3,3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,0.000033,0.999967,1
4,4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,0.000090,0.999910,1
...,...,...,...,...,...,...,...
322306,322306,抓住国家实施区域协调发展战略机遇，争取把白齐兴跨省生态经济合作区上升为国家战略，加强省际协商...,吉林省白城市,2018,0.000801,0.999199,1
322307,322307,（4）深入推进文化惠民工程，滑冰馆、体育场、游泳馆投入使用，贫困村综合性文化服务中心实现全覆盖,吉林省白城市,2018,0.009073,0.990927,1
322308,322308,（5）加大义务教育薄弱学校改造，建设标准化学校5所，实施校舍取暖“煤改电”,吉林省白城市,2018,0.046606,0.953394,1
322309,322309,（6）优化、加密公交线路，新区客运换乘中心投入使用；修建农村公路200公里，全面开工186个...,吉林省白城市,2018,0.027346,0.972654,1


In [14]:
import pandas as pd
import numpy as np
import json
import os
import six
from collections import Counter
from docx import Document
import os
import re
import pdb
import re
from random import choice
import random
import torch
from transformers import AutoTokenizer, BertModel
from datasets import Dataset
import torch
import time
torch.cuda.is_available()
from tqdm import tqdm

class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask, token_type_ids):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)

        out = self.fc(out.last_hidden_state[:, 0])
        out = out.softmax(dim=1)
        return out


tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
pretrained = BertModel.from_pretrained("bert-base-chinese")
model_for_test = Model()
model_for_test.load_state_dict(torch.load('models/model_state_dict_4epoch_4400iter.pkl'))
dataset_val = pd.read_csv("./0414res/test_dataset/test_dataset.csv", index_col=0)



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
dataset_val = dataset_val.head(1000)

In [16]:
device = torch.device("cuda")	
model_for_test = model_for_test.to(device)
pretrained = pretrained.to(device)

def get_lable_res(test_text):
    test_text = test_text[:510] # tokenizer 不提供截断padding接口，人为截断
    inputs = tokenizer(test_text, return_tensors="pt")
    res = model_for_test(input_ids=inputs["input_ids"].to(device), 
                        attention_mask=inputs["attention_mask"].to(device), 
                        token_type_ids=inputs["token_type_ids"].to(device))
    return res.cpu().detach().numpy()[0][0], res.cpu().detach().numpy()[0][1], res.argmax(dim=1).cpu().detach().numpy()[0]

tqdm.pandas(desc='set new lable')
a = time.time()
dataset_val["neg_prob"], dataset_val["pos_prob"], dataset_val["lable"] = zip(*dataset_val.progress_apply(lambda x: get_lable_res(x.text), axis=1))
print(time.time() - a, ": gpu 用时")

dataset_val
# dataset_val.to_csv("0414res/trained_res.csv", encoding="utf-8")

set new lable: 100%|██████████| 1000/1000 [00:07<00:00, 133.07it/s]

7.523104906082153 : gpu 用时


,text,city,year,neg_prob,pos_prob,lable
0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0.007142,0.992858,1
1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,0.015324,0.984676,1
2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,0.005322,0.994678,1
3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,0.000033,0.999967,1
4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,0.000090,0.999910,1
...,...,...,...,...,...,...
995,打好电网规划建设大会战，力争投产110千伏及以上输变电工程19项，进一步强化电力保障,广东省东莞市,2015,0.000440,0.999560,1
996,超前建设光纤、基站等通信设施，实现重点公共区域WiFi全覆盖,广东省东莞市,2015,0.012428,0.987572,1
997,启动深圳外环高速东莞段、环莞快速路三期等工程，加快虎门二桥等项目建设，推动石大公路大修等项目...,广东省东莞市,2015,0.000004,0.999996,1
998,圆满完成省下达我市的高速公路建设任务,广东省东莞市,2015,0.000639,0.999361,1


In [10]:
!pip install sentence_transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/bin/bash: /home/leo/anaconda3/envs/ernie/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.1 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 13.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 4.9 MB/s eta 0:00:0000:0100:02
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=0e52293f3278ab2d96093b717afc124aab2df25083a545f67cdb80beca8eca21
  S

In [38]:
from paddlenlp import Taskflow
similarity = Taskflow("text_similarity")
res = similarity([["春天适合种什么花？", "春天适合种什么菜？"]])
# [{'text1': '春天适合种什么花？', 'text2': '春天适合种什么菜？', 'similarity': 0.83402544}]
res[0]["similarity"]

[2023-07-09 17:22:24,428] [    INFO] - We are using <class 'paddlenlp.transformers.bert.tokenizer.BertTokenizer'> to load 'simbert-base-chinese'.
[2023-07-09 17:22:24,429] [    INFO] - Already cached /home/leo/.paddlenlp/models/simbert-base-chinese/vocab.txt
[2023-07-09 17:22:24,434] [    INFO] - tokenizer config file saved in /home/leo/.paddlenlp/models/simbert-base-chinese/tokenizer_config.json
[2023-07-09 17:22:24,434] [    INFO] - Special tokens file saved in /home/leo/.paddlenlp/models/simbert-base-chinese/special_tokens_map.json


0.8340253829956055

In [19]:
def get_aspect_lable(test_text):
    test_text = test_text[:510] # tokenizer 不提供截断padding接口，人为截断
    jinji = similarity([[test_text, "经济"]])
    shehui = similarity([[test_text, "社会"]])
    zhenzhi = similarity([[test_text, "政治"]])
    return jinji, shehui, zhenzhi
dataset_val["jinji"], dataset_val["shehui"], dataset_val["zhenzhi"] = zip(*dataset_val.progress_apply(lambda x: get_aspect_lable(x.text), axis=1))
dataset_val

set new lable: 100%|██████████| 1000/1000 [00:56<00:00, 17.71it/s]


,text,city,year,neg_prob,pos_prob,lable,jinji,shehui,zhenzhi
0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0.007142,0.992858,1,[{'text1': '预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一...,[{'text1': '预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一...,[{'text1': '预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一...
1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,0.015324,0.984676,1,[{'text1': '云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产...,[{'text1': '云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产...,[{'text1': '云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产...
2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,0.005322,0.994678,1,[{'text1': '积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城...,[{'text1': '积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城...,[{'text1': '积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城...
3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,0.000033,0.999967,1,[{'text1': '成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）...,[{'text1': '成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）...,[{'text1': '成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）...
4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,0.000090,0.999910,1,[{'text1': '大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养...,[{'text1': '大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养...,[{'text1': '大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养...
...,...,...,...,...,...,...,...,...,...
995,打好电网规划建设大会战，力争投产110千伏及以上输变电工程19项，进一步强化电力保障,广东省东莞市,2015,0.000440,0.999560,1,[{'text1': '打好电网规划建设大会战，力争投产110千伏及以上输变电工程19项，进...,[{'text1': '打好电网规划建设大会战，力争投产110千伏及以上输变电工程19项，进...,[{'text1': '打好电网规划建设大会战，力争投产110千伏及以上输变电工程19项，进...
996,超前建设光纤、基站等通信设施，实现重点公共区域WiFi全覆盖,广东省东莞市,2015,0.012428,0.987572,1,"[{'text1': '超前建设光纤、基站等通信设施，实现重点公共区域WiFi全覆盖', '...","[{'text1': '超前建设光纤、基站等通信设施，实现重点公共区域WiFi全覆盖', '...","[{'text1': '超前建设光纤、基站等通信设施，实现重点公共区域WiFi全覆盖', '..."
997,启动深圳外环高速东莞段、环莞快速路三期等工程，加快虎门二桥等项目建设，推动石大公路大修等项目...,广东省东莞市,2015,0.000004,0.999996,1,[{'text1': '启动深圳外环高速东莞段、环莞快速路三期等工程，加快虎门二桥等项目建设...,[{'text1': '启动深圳外环高速东莞段、环莞快速路三期等工程，加快虎门二桥等项目建设...,[{'text1': '启动深圳外环高速东莞段、环莞快速路三期等工程，加快虎门二桥等项目建设...
998,圆满完成省下达我市的高速公路建设任务,广东省东莞市,2015,0.000639,0.999361,1,"[{'text1': '圆满完成省下达我市的高速公路建设任务', 'text2': '经济'...","[{'text1': '圆满完成省下达我市的高速公路建设任务', 'text2': '社会'...","[{'text1': '圆满完成省下达我市的高速公路建设任务', 'text2': '政治'..."


In [21]:
all_res = pd.read_csv("0414res/trained_res.csv", index_col=0)
all_res

,text,city,year,neg_prob,pos_prob,lable
0,预计全年实现地区生产总值710.07亿元，比上年增长8.5%；地方一般公共预算收入58.7亿...,广东省云浮市,2016,0.007142,0.992858,1
1,云计算和信息服务产业发展取得实质性突破，编制实施《云计算及信息服务产业发展规划》和《“智慧云...,广东省云浮市,2016,0.015324,0.984676,1
2,积极发展循环经济和新能源汽车、新材料产业，罗定成为国家循环经济示范城市（县）建设地区，郁南虎...,广东省云浮市,2016,0.005322,0.994678,1
3,成功举办了第十二届中国（云浮）国际石材科技展览会暨第六届中国（云浮）石文化节、广东云浮·罗定稻米节,广东省云浮市,2016,0.000033,0.999967,1
4,大力建设“全国智慧旅游试点市”，以广东禅文化创意产业园为代表的健康养生旅游产业加快推进,广东省云浮市,2016,0.000090,0.999910,1
...,...,...,...,...,...,...
322306,抓住国家实施区域协调发展战略机遇，争取把白齐兴跨省生态经济合作区上升为国家战略，加强省际协商...,吉林省白城市,2018,0.000801,0.999199,1
322307,（4）深入推进文化惠民工程，滑冰馆、体育场、游泳馆投入使用，贫困村综合性文化服务中心实现全覆盖,吉林省白城市,2018,0.009073,0.990927,1
322308,（5）加大义务教育薄弱学校改造，建设标准化学校5所，实施校舍取暖“煤改电”,吉林省白城市,2018,0.046606,0.953394,1
322309,（6）优化、加密公交线路，新区客运换乘中心投入使用；修建农村公路200公里，全面开工186个...,吉林省白城市,2018,0.027346,0.972654,1


In [22]:
for i in all_res.groupby(["city", "year"]):
	print(i)
 
	break

(('上海市', 2007),                                                     text city  year  neg_prob   
71554  推进现代服务业集聚区建设，金融、现代物流、信息服务、旅游会展等现代服务业呈加快发展态势，第三...  上海市  2007  0.003161  \
71555   继续加大房地产市场调控，房地产开发投资增幅比上年有所回落，新建商品住房销售价格比上年下降3.2%  上海市  2007  0.586672   
71556  贯彻市人大常委会《关于进一步加强节约能源工作的决定》，吸收市政协《上海市能源发展形势分析和若...  上海市  2007  0.005873   
71557  第三轮环保三年行动计划项目加快推进，环保投入相当于市生产总值的比例达到3.02%，二氧化硫和...  上海市  2007  0.002323   
71558  贯彻全国科技大会精神，制订了《上海中长期科学和技术发展规划纲要》，出台“36条”配套政策及部...  上海市  2007  0.002119   
...                                                  ...  ...   ...       ...   
71628  必须进一步理顺政府与企业、市场、社会的关系，推进政府职能转变，在更好地履行经济调节、市场监管...  上海市  2007  0.551789   
71629                                　　　　注重加强社会管理和公共服务职能  上海市  2007  0.001648   
71630  优化公共资源配置，把政府工作的重点、政策支持的重点、财力保障的重点逐步向社会事业、公共事业发...  上海市  2007  0.687276   
71631                            继续推进文化市场、城市管理、经济领域的综合执法  上海市  2007  0.003452   
71632  同时，发挥政府内部监督机制的作用，加大执法监察、效能监察和廉政监察的力度，加大对重点领域、重...  上海市  2007  0.004526   

       pos_

In [52]:
demo = all_res

def get_single_res(test_text):
    test_text = test_text[:510] # tokenizer 不提供截断padding接口，人为截断
    jinji = similarity([[test_text, "经济"]])[0]["similarity"]
    shehui = similarity([[test_text, "社会"]])[0]["similarity"]
    zhenzhi = similarity([[test_text, "政治"]])[0]["similarity"]
    shentai = similarity([[test_text, "生态文明"]])[0]["similarity"]
    wenhua = similarity([[test_text, "文化"]])[0]["similarity"]
    total_res = jinji + shehui + zhenzhi + shentai + wenhua + 5
    return (jinji + 1) / total_res, (shehui + 1) / total_res, (zhenzhi + 1) / total_res, (shentai + 1) / total_res, (wenhua + 1) / total_res

def get_aspect_lable(x):
    if x.shape[0] > 20:
        x = x.sample(10)
    x["jinji"], x["shehui"], x["zhenzhi"], x["shentai"], x["wenhua"] = zip(*x.apply(lambda x: get_single_res(x.text), axis=1))
    return x

i = 0
city_list = []
year_list = []
jinji_list = []
shehui_list = []
zhenzhi_list = []
shentai_list = []
wenhua_list = []
for k, v in demo.groupby(["city", "year"]):
    v = get_aspect_lable(v)
    v["jinji_res"] = v["pos_prob"] * v["jinji"]
    v["shehui_res"] = v["pos_prob"] * v["shehui"]
    v["zhenzhi_res"] = v["pos_prob"] * v["zhenzhi"]
    v["shentai_res"] = v["pos_prob"] * v["shentai"]
    v["wenhua_res"] = v["pos_prob"] * v["wenhua"]
    # print(k[0], k[1], v["jinji_res"].mean(), v["shehui_res"].mean(), v["zhenzhi_res"].mean(), v["shentai_res"].mean(), v["wenhua_res"].mean())
    # print(v)
    city_list.append(k[0])
    year_list.append(k[1])
    jinji_list.append(v["jinji_res"].mean())
    shehui_list.append(v["shehui_res"].mean())
    zhenzhi_list.append(v["zhenzhi_res"].mean())
    shentai_list.append(v["shentai_res"].mean())
    wenhua_list.append(v["wenhua_res"].mean())
    if i % 500 == 0:
        print(i)
        res_dict = {'city':city_list,
            'year': year_list,
            'jinji': jinji_list,
            'shehui': shehui_list,
            'zhenzhi': zhenzhi_list,
            'shentai': shentai_list,
            'wenhua':wenhua_list}
        res_df = pd.DataFrame(res_dict)
        res_df.to_csv("0707_res.csv", encoding='utf-8')
    i = i + 1
    # break
res_dict = {'city':city_list,
            'year': year_list,
            'jinji': jinji_list,
            'shehui': shehui_list,
            'zhenzhi': zhenzhi_list,
            'shentai': shentai_list,
            'wenhua':wenhua_list}
res_df = pd.DataFrame(res_dict)
res_df.to_csv("0707_res.csv", encoding='utf-8')
res_df

0
500
1000
1500
2000
2500
3000
3500
4000


,city,year,jinji,shehui,zhenzhi,shentai,wenhua
0,上海市,2007,0.181264,0.166566,0.165470,0.191787,0.179108
1,上海市,2008,0.140076,0.136498,0.130090,0.135341,0.131654
2,上海市,2009,0.168618,0.156309,0.147943,0.169309,0.161771
3,上海市,2010,0.178346,0.163836,0.157805,0.181059,0.164465
4,上海市,2011,0.158890,0.148717,0.144343,0.165402,0.153529
...,...,...,...,...,...,...,...
4233,黑龙江省齐齐哈尔市,2017,0.195544,0.185076,0.179093,0.206823,0.204379
4234,黑龙江省齐齐哈尔市,2018,0.179538,0.169901,0.165820,0.196348,0.185318
4235,黑龙江省齐齐哈尔市,2019,0.197845,0.181531,0.174693,0.207448,0.190613
4236,黑龙江省齐齐哈尔市,2020,0.195013,0.182860,0.177379,0.213570,0.191879


In [53]:
import pandas as pd
res = pd.read_csv("0716-五位一体关键词.csv")
res

,General index,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,主题,经济,社会,政治,生态文明,文化
1,关键词,经济、金融、产业、贸易、进口、出口、工业、投资、消费、招商、营商、引资、企业、制造、产值、工...,民生、教育、体育、娱乐、科技、就业、失业、卫生、医疗、医院、学校、公园、中学、小学、幼儿园、...,政治、制度、体制、基层、党、政府机构、政府职能、行政、人大、政协、监督、监察、腐败、反腐、执...,污染、污水、绿色、低碳、节能、减排、降耗、生态、垃圾、清洁、环境、环保、空气质量、水质、废物...,文化、价值观、价值体系、软实力、思想、诚信、信仰、宣传、道德、美德、文明、自由、平等、公正、...


In [54]:
res.columns = ["1", "经济", "社会", "政治", "生态文明", "文化"]
res

,1,经济,社会,政治,生态文明,文化
0,主题,经济,社会,政治,生态文明,文化
1,关键词,经济、金融、产业、贸易、进口、出口、工业、投资、消费、招商、营商、引资、企业、制造、产值、工...,民生、教育、体育、娱乐、科技、就业、失业、卫生、医疗、医院、学校、公园、中学、小学、幼儿园、...,政治、制度、体制、基层、党、政府机构、政府职能、行政、人大、政协、监督、监察、腐败、反腐、执...,污染、污水、绿色、低碳、节能、减排、降耗、生态、垃圾、清洁、环境、环保、空气质量、水质、废物...,文化、价值观、价值体系、软实力、思想、诚信、信仰、宣传、道德、美德、文明、自由、平等、公正、...


In [55]:
# jinji = res["经济"].to_list()
# jinji[1] = jinji[1].replace("'","")
# new_res = jinji[1].split("、")
# new_res[3]

theme_list = ["经济", "社会", "政治", "生态文明", "文化"]
theme_dict = {}

for theme in theme_list:
    single_theme_list = res[theme].to_list()[1].split("、")
    theme_dict[theme] = single_theme_list
print(theme_dict)





{'经济': ['经济', '金融', '产业', '贸易', '进口', '出口', '工业', '投资', '消费', '招商', '营商', '引资', '企业', '制造', '产值', '工业增加值', '利润', '生产', '市场', '服务业', '商业'], '社会': ['民生', '教育', '体育', '娱乐', '科技', '就业', '失业', '卫生', '医疗', '医院', '学校', '公园', '中学', '小学', '幼儿园', '大学', '高校', '养老', '老旧小区', '保障性住房', '棚改', '社区', '社会救助', '社会保障', '社保', '医保', '保险', '公共服务', '消防', '应急', '治安', '平安', '社会治理', '安全', '获得感', '幸福', '服务型政府', '政府服务', '社会和谐', '社会矛盾', '慈善'], '政治': ['政治', '制度', '体制', '基层', '党', '政府机构', '政府职能', '行政', '人大', '政协', '监督', '监察', '腐败', '反腐', '执法', '施政', '公务员', '廉洁', '廉政', '作风', '四风', '民主集中', '违法', '违纪', '政务', '公信', '简政', '放权', '法制', '法治', '机构改革', '形式主义', '官僚主义', '意识形态'], '生态文明': ['污染', '污水', '绿色', '低碳', '节能', '减排', '降耗', '生态', '垃圾', '清洁', '环境', '环保', '空气质量', '水质', '废物', '回收', '水土流失', '耗能', '绿水青山'], '文化': ['文化', '价值观', '价值体系', '软实力', '思想', '诚信', '信仰', '宣传', '道德', '美德', '文明', '自由', '平等', '公正', '爱国', '敬业', '诚信', '友善', '富强', '民主', '和谐', '哲学', '社会科学', '中国梦', '凝聚力']}


In [56]:
columns_list = []
columns_list = ['year', "city"] + theme_list + theme_dict["经济"] + \
    theme_dict["社会"] + theme_dict["政治"] + theme_dict["生态文明"] + theme_dict["文化"]
res_table = pd.DataFrame(columns=columns_list)
res_table

,year,city,经济,社会,政治,生态文明,文化,经济,金融,产业,...,敬业,诚信,友善,富强,民主,和谐,哲学,社会科学,中国梦,凝聚力


In [58]:
# 年份，城市，a词，b词，c词
import os
path1 = './地级工作报告(2007-2021)' #待读取文件的文件夹绝对地址
# path1 = './demo_地级市工作报告' #待读取文件的文件夹绝对地址
files = os.listdir(path1) #获得文件夹中所有文件的名称列表
test_dataset = pd.DataFrame(
  columns=['text','city','year']
) 

length_list = [0, 21, 21+41, 21+41+34, 21+41+34+19]
total_length = len(theme_dict["经济"]) + len(theme_dict["社会"]) + len(theme_dict["政治"]) + len(theme_dict["生态文明"]) + len(theme_dict["文化"])
for file in files:
    try:
        city_name = file[:-9]
        year_name = int(file[-9:-5])
        path = path1 +  '/' + file
        document = Document(path)
        sentences = document.paragraphs[0].text.strip().replace('\n','').split('。')
        
        small_list = [0] * 5
        long_list = [0] * total_length
        
        for sentence in sentences:
            for i in range(len(theme_list)):
                single_theme = theme_list[i]
                for j in range(len(theme_dict[single_theme])):
                    small_theme = theme_dict[single_theme][j]  # 第i个大主题下的第j个小主题词
                    k = length_list[i] + j
                    if small_theme in sentence:
                        small_list[i] = small_list[i] + 1
                        long_list[k] = long_list[k] + 1
                        
        single_case_list = [year_name, city_name] + small_list + long_list
        # print(small_list)
        # print(long_list)
        res_table.loc[len(res_table.index)] = single_case_list
            # break
    except:
        pass 
res_table.to_csv('0717res.csv', encoding='utf-8')
# test_dataset.to_csv("0414res/test_dataset/test_dataset.csv", encoding="utf-8")

In [57]:
total_length = len(theme_dict["经济"]) + len(theme_dict["社会"]) + len(theme_dict["政治"]) + len(theme_dict["生态文明"]) + len(theme_dict["文化"])
print(len(theme_dict["经济"]))
print(len(theme_dict["社会"]))
print(len(theme_dict["政治"]))
print(len(theme_dict["生态文明"]))
print(len(theme_dict["文化"]))



21
41
34
19
25


In [67]:
res_table

,year,city,经济,社会,政治,生态文明,文化,经济,金融,产业,...,敬业,诚信,友善,富强,民主,和谐,哲学,社会科学,中国梦,凝聚力
0,2016,广东省云浮市,222,177,116,73,34,25,14,54,...,0,0,0,0,4,3,0,0,0,0
1,2018,安徽省阜阳市,161,110,95,71,40,25,6,37,...,0,1,0,0,4,2,0,0,0,0
2,2012,湖北省黄石市,252,131,93,57,43,38,5,47,...,0,0,0,0,3,4,0,0,0,0
3,2013,湖南省益阳市,226,143,97,78,58,25,9,36,...,0,1,0,0,6,6,0,1,0,0
4,2016,辽宁省辽阳市,246,131,101,47,57,36,7,38,...,0,3,0,0,7,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,2007,福建省福州市,197,152,114,54,71,29,4,21,...,0,1,0,0,11,16,0,1,0,0
4234,2011,安徽省合肥市,224,173,109,61,36,32,9,37,...,0,0,0,0,5,4,0,0,0,0
4235,2012,山西省晋中市,194,115,92,57,31,26,3,35,...,0,0,0,0,8,2,0,0,0,0
4236,2009,四川省泸州市,157,79,82,39,22,21,8,21,...,0,0,0,0,7,4,0,0,0,0


In [64]:
res_table["文化"].mean()

文化    36.332704
文化    12.878244
dtype: float64

In [65]:
total = (191.779849 + 116.41647003303446 + 85.240916 + 57.59344030202926 + 36.332704) / 5
jijin_score = 191 / total
shehui_score = 116 / total
zhengzhi_score = 85 / total
shentai_score = 57 / total
wenhua_score = 36 / total


In [66]:
score_table = pd.read_csv("0707_res.csv", encoding="utf-8", index_col=0)
score_table

,city,year,jinji,shehui,zhenzhi,shentai,wenhua
0,上海市,2007,0.181264,0.166566,0.165470,0.191787,0.179108
1,上海市,2008,0.140076,0.136498,0.130090,0.135341,0.131654
2,上海市,2009,0.168618,0.156309,0.147943,0.169309,0.161771
3,上海市,2010,0.178346,0.163836,0.157805,0.181059,0.164465
4,上海市,2011,0.158890,0.148717,0.144343,0.165402,0.153529
...,...,...,...,...,...,...,...
4233,黑龙江省齐齐哈尔市,2017,0.195544,0.185076,0.179093,0.206823,0.204379
4234,黑龙江省齐齐哈尔市,2018,0.179538,0.169901,0.165820,0.196348,0.185318
4235,黑龙江省齐齐哈尔市,2019,0.197845,0.181531,0.174693,0.207448,0.190613
4236,黑龙江省齐齐哈尔市,2020,0.195013,0.182860,0.177379,0.213570,0.191879


In [75]:
weight_table = res_table[["year", "city", "经济", "社会", "政治", "生态文明", "文化"]]
weight_table.columns = ["year", "city", "经济", "经济1", "社会", "政治", "政治1", "生态文明", "文化", "文化1"]
weight_table = weight_table[["year", "city", "经济", "社会", "政治", "生态文明", "文化"]]
weight_table

,year,city,经济,社会,政治,生态文明,文化
0,2016,广东省云浮市,222,177,116,73,34
1,2018,安徽省阜阳市,161,110,95,71,40
2,2012,湖北省黄石市,252,131,93,57,43
3,2013,湖南省益阳市,226,143,97,78,58
4,2016,辽宁省辽阳市,246,131,101,47,57
...,...,...,...,...,...,...,...
4233,2007,福建省福州市,197,152,114,54,71
4234,2011,安徽省合肥市,224,173,109,61,36
4235,2012,山西省晋中市,194,115,92,57,31
4236,2009,四川省泸州市,157,79,82,39,22


In [78]:
all_table = pd.merge(score_table, weight_table, on=['year','city'],how='inner')

(191.779849 + 116.41647003303446 + 85.240916 + 57.59344030202926 + 36.332704) / 5


all_table["经济1"] = all_table["jinji"] * jijin_score * all_table["经济"] / 191
all_table["社会1"] = all_table["shehui"] * shehui_score * all_table["社会"] / 116
all_table["政治1"] = all_table["zhenzhi"] * zhengzhi_score * all_table["政治"] / 85
all_table["生态文明1"] = all_table["shentai"] * shentai_score * all_table["生态文明"] / 57
all_table["文化1"] = all_table["wenhua"] * wenhua_score * all_table["文化"] / 36
all_table

,city,year,jinji,shehui,zhenzhi,shentai,wenhua,经济,社会,政治,生态文明,文化,经济1,社会1,政治1,生态文明1,文化1
0,上海市,2007,0.181264,0.166566,0.165470,0.191787,0.179108,226,186,197,67,82,0.420278,0.317845,0.334428,0.131829,0.150677
1,上海市,2008,0.140076,0.136498,0.130090,0.135341,0.131654,231,217,170,70,88,0.331966,0.303881,0.226888,0.097195,0.118860
2,上海市,2009,0.168618,0.156309,0.147943,0.169309,0.161771,240,199,163,61,43,0.415176,0.319120,0.247400,0.105956,0.071365
3,上海市,2010,0.178346,0.163836,0.157805,0.181059,0.164465,218,212,166,66,52,0.398876,0.356339,0.268749,0.122597,0.087739
4,上海市,2011,0.158890,0.148717,0.144343,0.165402,0.153529,262,220,179,83,72,0.427086,0.335660,0.265073,0.140843,0.113407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,黑龙江省齐齐哈尔市,2017,0.195544,0.185076,0.179093,0.206823,0.204379,211,100,77,50,28,0.423297,0.189875,0.141477,0.106093,0.058710
4234,黑龙江省齐齐哈尔市,2018,0.179538,0.169901,0.165820,0.196348,0.185318,187,95,101,85,27,0.344441,0.165591,0.171821,0.171223,0.051333
4235,黑龙江省齐齐哈尔市,2019,0.197845,0.181531,0.174693,0.207448,0.190613,204,98,79,103,28,0.414069,0.182513,0.141586,0.219212,0.054756
4236,黑龙江省齐齐哈尔市,2020,0.195013,0.182860,0.177379,0.213570,0.191879,229,109,94,76,17,0.458159,0.204485,0.171060,0.166522,0.033465


In [80]:
all_table.to_csv("0718res.csv", encoding="utf-8")
final_table = all_table[["year", "city", "经济1", "社会1", "政治1", "生态文明1", "文化1"]]
final_table.cloumns = ["year", "city", "经济", "社会", "政治", "生态文明", "文化"]
final_table.to_csv("0719res.csv", encoding="utf-8")
final_table


/tmp/ipykernel_4911/2906168234.py:3: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  final_table.cloumns = ["year", "city", "经济", "社会", "政治", "生态文明", "文化"]


,year,city,经济1,社会1,政治1,生态文明1,文化1
0,2007,上海市,0.420278,0.317845,0.334428,0.131829,0.150677
1,2008,上海市,0.331966,0.303881,0.226888,0.097195,0.118860
2,2009,上海市,0.415176,0.319120,0.247400,0.105956,0.071365
3,2010,上海市,0.398876,0.356339,0.268749,0.122597,0.087739
4,2011,上海市,0.427086,0.335660,0.265073,0.140843,0.113407
...,...,...,...,...,...,...,...
4233,2017,黑龙江省齐齐哈尔市,0.423297,0.189875,0.141477,0.106093,0.058710
4234,2018,黑龙江省齐齐哈尔市,0.344441,0.165591,0.171821,0.171223,0.051333
4235,2019,黑龙江省齐齐哈尔市,0.414069,0.182513,0.141586,0.219212,0.054756
4236,2020,黑龙江省齐齐哈尔市,0.458159,0.204485,0.171060,0.166522,0.033465


In [83]:
final_table.cloumns = ["year", "city", "经济", "社会", "政治", "生态文明", "文化"]

In [84]:
final_table

,year,city,经济1,社会1,政治1,生态文明1,文化1
0,2007,上海市,0.420278,0.317845,0.334428,0.131829,0.150677
1,2008,上海市,0.331966,0.303881,0.226888,0.097195,0.118860
2,2009,上海市,0.415176,0.319120,0.247400,0.105956,0.071365
3,2010,上海市,0.398876,0.356339,0.268749,0.122597,0.087739
4,2011,上海市,0.427086,0.335660,0.265073,0.140843,0.113407
...,...,...,...,...,...,...,...
4233,2017,黑龙江省齐齐哈尔市,0.423297,0.189875,0.141477,0.106093,0.058710
4234,2018,黑龙江省齐齐哈尔市,0.344441,0.165591,0.171821,0.171223,0.051333
4235,2019,黑龙江省齐齐哈尔市,0.414069,0.182513,0.141586,0.219212,0.054756
4236,2020,黑龙江省齐齐哈尔市,0.458159,0.204485,0.171060,0.166522,0.033465
